In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df_train = pd.read_csv(r"train.csv")
df_test = pd.read_csv(r"test.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#replace Nan values with 0s for the Age column 

df_train['Age'].fillna(0, inplace=True)
df_train['Embarked'].fillna('S', inplace=True)

df_test['Age'].fillna(0, inplace=True)
df_test['Embarked'].fillna('S', inplace=True)


# **Assume all women survived**

In [28]:
women = df_train.loc[df_train.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived: ", rate_women)

% of women who survived:  0.7420382165605095


In [29]:
men = df_train.loc[df_train.Sex == 'male']['Survived']
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


# **Random forest model**

In [30]:
from sklearn.ensemble import RandomForestClassifier

y = df_train['Survived']

feature = ['Pclass','Sex','Age','SibSp','Parch']

X = pd.get_dummies(df_train[feature])
X_test = pd.get_dummies(df_test[feature])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

model.fit(X,y)

predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerID': df_test.PassengerId, 'Survived': predictions})

output.to_csv('submission2.csv',index=False) 

# **Logist Regression Model**

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import MinMaxScaler

In [7]:
y = df_train['Survived']

feature = ['Pclass','Sex','Age','SibSp','Parch','Embarked']

#X = pd.get_dummies(df_train[feature])
#X = pd.get_dummies(df_test[feature])


In [14]:
scaler = MinMaxScaler(feature_range = (0,1))

scaler.fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)

NameError: name 'X_train' is not defined

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df_train['Embarked'] = label_encoder.fit_transform(df_train['Embarked'])

In [9]:
parameters_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}


model = LogisticRegression()


# grid search 
grid_search = GridSearchCV(model, parameters_grid, cv=5)

grid_search.fit(X,y)

best_C = grid_search.best_params_['C']

c:\Users\hpalm\anaconda3\envs\tfradeon\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\Users\hpalm\anaconda3\envs\tfradeon\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

In [11]:
model = LogisticRegression(C=best_C)

model.fit(X,y)

predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerID': df_test.PassengerId, 'Survived': predictions})

In [12]:
output.to_csv('submission_L.csv',index=False) 